In [47]:
import sys
sys.path.append("../script")
from data_preparation import *
import pandas as pd
from patsy import dmatrices
import statsmodels.formula.api as smf

In [2]:
train = pd.read_csv("../data/application_train_vf.csv")
test = pd.read_csv("../data/application_test_vf.csv")

In [3]:
dataprep = DataPreparation(train, test, 0.3)

In [4]:
train_prepared, test_prepared = dataprep.get_prepared_data()

Type des variables convertis ✅
Valeurs manquantes traitées ✅
Discrétisation des variables numériques en cours ... 


100%|██████████| 12/12 [05:36<00:00, 28.05s/it]

Variables numériques discrétisées ✅
Discrétisation des variables catégorielles en cours ... 
Variables catégorielles discrétisées ✅


In [8]:
df_train = train_prepared.copy()

In [50]:
df_train

,TARGET,date_mensuelle,DAYS_BIRTH_disc_int,EXT_SOURCE_2_disc_int,AMT_GOODS_PRICE_disc_int,DAYS_REGISTRATION_disc_int,DAYS_LAST_PHONE_CHANGE_disc_int,EXT_SOURCE_3_disc_int,AMT_CREDIT_disc_int,AMT_ANNUITY_disc_int,...,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,FLAG_WORK_PHONE,FLAG_PHONE,LIVE_CITY_NOT_WORK_CITY,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_HOUSING_TYPE
0,0,2013-01-01,[-13434--7489],[0.46-0.85],[454500.0-4050000.0],[-6206.0-0.0],[-1021.0-0.0],[0.27-0.9],[675000.0-4050000.0],[1615.5-41625.0],...,3,3,0,1,0,Cash loans,M,Y,Y,House / apartment
1,0,2013-01-01,[-13434--7489],[0.46-0.85],[40500.0-453465.0],[-6206.0-0.0],[-1021.0-0.0],[0.27-0.9],[45000.0-674842.5],[1615.5-41625.0],...,3,3,0,1,0,Cash loans,F,N,Y,With parents
2,0,2013-01-01,[-13434--7489],[0.46-0.85],[40500.0-453465.0],[-24672.0--6207.0],[-1021.0-0.0],[0.27-0.9],[45000.0-674842.5],[1615.5-41625.0],...,2,2,1,0,0,Cash loans,F,Y,N,House / apartment
3,0,2013-01-01,[-19539--13435],[0.46-0.85],[454500.0-4050000.0],[-24672.0--6207.0],[-1021.0-0.0],[0.27-0.9],[675000.0-4050000.0],[1615.5-41625.0],...,1,1,0,0,0,Cash loans,F,Y,Y,House / apartment
4,0,2013-01-01,[-19539--13435],[0.0-0.16],[40500.0-453465.0],[-6206.0-0.0],[-4292.0--1022.0],[0.27-0.9],[45000.0-674842.5],[1615.5-41625.0],...,2,2,0,1,1,Cash loans,F,N,N,House / apartment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279995,1,2020-04-01,[-19539--13435],[0.46-0.85],[454500.0-4050000.0],[-24672.0--6207.0],[-1021.0-0.0],[0.27-0.9],[675000.0-4050000.0],[1615.5-41625.0],...,2,2,0,0,0,Cash loans,F,N,Y,House / apartment
279996,1,2020-04-01,[-19539--13435],[0.46-0.85],[454500.0-4050000.0],[-6206.0-0.0],[-4292.0--1022.0],[0.27-0.9],[675000.0-4050000.0],[1615.5-41625.0],...,2,2,0,0,0,Cash loans,F,N,N,House / apartment
279997,1,2020-04-01,[-13434--7489],[0.46-0.85],[454500.0-4050000.0],[-6206.0-0.0],[-4292.0--1022.0],[0.27-0.9],[675000.0-4050000.0],[41629.5-258025.5],...,2,2,0,0,0,Cash loans,F,Y,Y,House / apartment
279998,1,2020-04-01,[-13434--7489],[0.16-0.46],[40500.0-453465.0],[-6206.0-0.0],[-4292.0--1022.0],[0.27-0.9],[45000.0-674842.5],[1615.5-41625.0],...,2,2,0,1,1,Cash loans,F,N,Y,Rented apartment


# Modélisation

In [23]:
df_validation = df_train.iloc[280000:,:]
df_train = df_train.iloc[:280000,:]

In [19]:
df_train["TARGET"] = df_train["TARGET"].astype("int")

In [84]:
def calculate_percentage_default(row, data_frame):
    variable = row['Variable']
    modality = row['Modality']
    if modality.isdigit() :
        modality = int(modality)

    default_count = data_frame[data_frame[variable] == modality]["TARGET"].sum()
    total_count = data_frame.shape[0]
    return round((default_count / total_count)*100,2)

def calculate_pcentage_class(row, data_frame):
    variable = row['Variable']
    modality = row['Modality']
    if modality.isdigit() :
        modality = int(modality)

    default_count = data_frame[data_frame[variable] == modality].shape[0]
    total_count = data_frame.shape[0]
    return round((default_count / total_count)*100,2)

In [88]:
formula = 'TARGET ~ C(FLAG_EMP_PHONE) + C(REG_CITY_NOT_LIVE_CITY) + C(EXT_SOURCE_2_disc_int)'
model = smf.logit(formula=formula, data=df_train)
logit_model = model.fit()

results_summary_frame = logit_model.summary2().tables[1]

coefs = results_summary_frame['Coef.']
p_values = results_summary_frame['P>|z|']

max_coef = coefs.loc[coefs.index != 'Intercept'].max()
min_coef = coefs.loc[coefs.index != 'Intercept'].min()

score_card = pd.DataFrame(columns=['Variable', 'Coefficient', 'P-Value', 'Score'])

for variable in coefs.index[1:]:  
    coef = round(coefs[variable],2)
    p_value = round(p_values[variable],4)
    score = round(abs(max_coef - coef) / (max_coef - min_coef) * 1000, 2)
    
    score_card.loc[len(score_card)] = [variable, coef, p_value, score]

score_card['Normalized Score'] = round((score_card['Score'] - score_card['Score'].min()) / (score_card['Score'].max() - score_card['Score'].min()) * 1000,2)

score_card["Modality"] = score_card["Variable"].apply(lambda x: x.split("[T.")[1][:-1])
score_card["Variable"] = score_card["Variable"].apply(lambda x: (x.split("C(")[1]).split(")")[0])

score_card["Pcentage_Défaut"] = score_card.apply(lambda row: calculate_percentage_default(row, df_train), axis=1)
score_card["Pcentage_Classe"] = score_card.apply(lambda row: calculate_pcentage_class(row, df_train), axis=1)


score_card = score_card[['Variable', "Modality", 'Coefficient', 'P-Value', 'Normalized Score', "Pcentage_Défaut", "Pcentage_Classe"]]

score_card

Optimization terminated successfully.
         Current function value: 0.257106
         Iterations 7


,Variable,Modality,Coefficient,P-Value,Normalized Score,Pcentage_Défaut,Pcentage_Classe
0,FLAG_EMP_PHONE,1,0.51,0.0,0.00,6.65,81.94
1,REG_CITY_NOT_LIVE_CITY,1,0.41,0.0,48.51,0.91,7.81
2,EXT_SOURCE_2_disc_int,[0.16-0.46],-0.72,0.0,617.39,2.73,25.60
3,EXT_SOURCE_2_disc_int,[0.46-0.85],-1.48,0.0,1000.00,3.62,68.30


In [57]:
import numpy as np

def calculate_contributions(N_scores, r_proportions, k_variables):
    """
    Calculate the contribution of modalities for each variable.

    Parameters:
    - N_scores: A dictionary where keys are variables and values are lists of tuples with the modality score and its average.
    - r_proportions: A dictionary where keys are variables and values are lists of the population proportion for each modality.
    - k_variables: The number of variables in the model.

    Returns:
    A dictionary with the contribution of each modality for each variable.
    """
    contributions = {}
    for variable, scores in N_scores.items():
        # Calculate average score for the variable
        N_bar = np.mean([score[0] for score in scores])

        # Calculate the numerator for the variable's contribution
        numerator = np.sqrt(sum([r_proportions[variable][j] * (score[0] - N_bar) ** 2 for j, score in enumerate(scores)]))
        
        # Calculate the denominator for the variable's contribution
        denominator = np.sqrt(sum([
            sum([r_proportions[var][j] * (score[0] - np.mean([s[0] for s in N_scores[var]])) ** 2
                 for j, score in enumerate(N_scores[var])])
            for var in N_scores
        ]))
        
        # Calculate contributions for each modality
        contributions[variable] = [numerator / denominator for score in scores]

    return contributions

# Example usage:
# Assuming you have calculated the N_scores and r_proportions for each variable
N_scores = {
    'variable1': [(100, 90), (200, 180)], 'variable2': [(30, 120), (100, 90)], # example (score, average score) for each modality
    # ... other variables
}
r_proportions = {
    'variable1': [0.5, 0.5], 'variable2' : [0.2, 0.8]  # example population proportion for each modality
    # ... other variables
}
k_variables = 2  # example number of variables

contributions = calculate_contributions(N_scores, r_proportions, k_variables)
print(contributions)


{'variable1': [0.8192319205190405, 0.8192319205190405], 'variable2': [0.5734623443633283, 0.5734623443633283]}
